In [1]:
from openai import OpenAI
import os
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
# from tqdm import tqdm
from tqdm.auto import tqdm  # for notebooks
import glob
import re 
import string
from FlagEmbedding import BGEM3FlagModel
import numpy as np
# doc = Document("/Users/kong/Documents/เอกสารประกาศ KB/25.ตรวจคัดกรองมะเร็งปากมดลูก/ตรวจคัดกรองมะเร็งปากมดลูก.docx")
tqdm.pandas()
os.environ["OPENTYPHOON_API_KEY"] = "add"
client = OpenAI(api_key=os.environ["OPENTYPHOON_API_KEY"], base_url="https://api.opentyphoon.ai/v1")
embed = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
def embed_encode(docs):
    return embed.encode(docs)['dense_vecs']

/Users/kong/Documents/mod/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kong/Documents/mod/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:
def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if type(item) == list:
            flattened.extend(item)
        else:
            flattened.append(item)
    return flattened

def clean(text):
  text = text.translate(str.maketrans('', '', '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~'''))
  text =text.lower()
  text = text.replace('\n', '')
  return text

df = pd.read_csv('/Users/kong/Documents/mod/RAG_CC67/folder_rag/chunk_data.csv')
df['Encode'] = df['Encode'].apply(lambda x: np.array([float(i) for i in x.split('|')]))
uni_id = list(df['Chunk_id'].unique())
merge_id = []
for i in uni_id:
    t = list(df[df['Chunk_id']==i].index)
    if len(t) <= 1:
        continue
    merge_id.extend(list(range(t[1],t[-1]+1,2)))

print(merge_id)

[1, 3, 5, 7, 9, 12, 14, 16, 18, 20, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143]


In [3]:
def sbert_search(query,chunk_list,df2):
    embed_q = embed_encode([clean(query)])[0]
    data = df2[df2['Chunk_id'].isin(chunk_list)]
    data['Score'] = data['Encode'].apply(lambda x: embed_q@x)
    data = data.sort_values(by='Score',ascending=False).head(30)
    id = list(data.index)
    n_id = flatten_list([[i-1, i+1] if i in merge_id else i for i in id])
    # return data['Chunk_text'][34]
    txt = []
    current =''
    # print(n_id)
    for i in n_id:
        current = str(df2['Chunk_text'][i])
        if i == 0:
            break
        else:
            pre = i-2
        for j in list(range(pre,-1,-2)):
            if pre in n_id:
                current = df2['Chunk_text'][pre] + current
                n_id.remove(pre)
            else:
                break
        if i == df2.shape[1]-1:
            break
        else:
            post = i+2
        for j in list(range(post,df2.shape[1],2)):
            if post in n_id:
                current = current + df2['Chunk_text'][post]
                n_id.remove(post)
            else:
                break
        txt.append(current)
    
    return txt


In [10]:
yn_prompt = '''
You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Query: {query}
Document: """{document}"""
Relevant:
'''

list_document ='DOCUMENT: """"{document}""" '

ans_prompt ='''
You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:

Keep your answer ground in the facts provided in DOCUMENT section.
If the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].

{template}

QUESTION: {question}
ANSWER:
'''

In [11]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def retrival_70b(query,document):
    response = client.chat.completions.create(
        model="typhoon-instruct",
        messages=[{"role": "user", "content": yn_prompt.format(query=query, document=document)}],
        max_tokens=1,
        temperature= 0,
        top_p = 0.9,
        seed=55
    )

    return response.choices[0].message.content

def clean_yn(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.translate(str.maketrans('', '', string.digits))
    return text.strip()

def answer(question,template):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": ans_prompt.format(question=question, template=template)}],
        max_tokens=4096,
        temperature= 0.3,
        top_p = 0.9,
        seed=55
    )

    return response.choices[0].message.content

def chat_rag(query):
    data = pd.read_csv('/Users/kong/Documents/mod/RAG_CC67/folder_rag/page_data.csv')
    # global temp
    f_yn = []
    for i in data['Passage']:
        f_yn.append(clean_yn(retrival_70b(query,i)))
    if 'yes' in f_yn:
        idx = [i for i,x in enumerate(f_yn) if x=='yes']
        choosen_chunk =  list(data['Chunk_id'][idx])
    else:
        return 'None'
    
    retrival_txt = sbert_search(query,choosen_chunk,df)
    text = []
    n_length = 0
    for i in retrival_txt:
        n_length += len(i)
        if n_length< 8000:
            text.append(list_document.format(document = i))
    template = '\n'.join(text) 
    ans = answer(query,template)
    if clean_yn(ans) == 'none':
        return 'None'
    else: 
        return (ans,list(data[data['Chunk_id'].isin(choosen_chunk)]['File_name']))
    

In [18]:
chat_rag(query= 'pep smear คืออะไร?')[0]

/var/folders/wr/xvr6b48d2c5_1lj5wh1h36mr0000gn/T/ipykernel_4889/3647027012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Score'] = data['Encode'].apply(lambda x: embed_q@x)


'วิธีการตรวจคัดกรองหาระยะก่อนเป็นมะเร็งปากมดลูก แพทย์จะใช้เครื่องมือสอดผ่านและถ่างช่องคลอด จากนั้นทําการป้ายเซลล์จากมดลูกส่งตรวจทางห้องปฏิบัติการเพื่อตรวจหาเซลล์ที่ผิดปกติ'

In [19]:
chat_rag(query= 'กลุ่มเสี่ยงสูงมะเร็ง?')[0]

/var/folders/wr/xvr6b48d2c5_1lj5wh1h36mr0000gn/T/ipykernel_4889/3647027012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Score'] = data['Encode'].apply(lambda x: embed_q@x)


'หญิงที่มีความเสี่ยงสูง เช่น มีคู่นอนหลายคน, มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย, หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย.'

In [26]:
chat_rag(query= 'พิการมีกี่ประเภท?')[0]

/var/folders/wr/xvr6b48d2c5_1lj5wh1h36mr0000gn/T/ipykernel_4889/3647027012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Score'] = data['Encode'].apply(lambda x: embed_q@x)


'N'

In [21]:
data = pd.read_csv('/Users/kong/Documents/mod/RAG_CC67/folder_rag/page_data.csv')
page = list(data.Passage)
query= 'pep smear ราคา?'
data_list = []
for i in page:
    data_list.append([query,i])
data_list[0]

['pep smear ราคา?',
 'การใช้สิทธิย่อยคนพิการ (ท74) 1.กลุ่มเป้าหมาย ผู้มีสิทธิหลักประกันสุขภาพแห่งชาติ ที่มีประเภทสิทธิย่อยคนพิการ (ท74) รวมถึงผู้มีสิทธิหลักประกันสุขภาพแห่งชาติ (ผู้ประกันตนคนพิการ : DIS) 2.สิทธิประโยชน์ที่ได้รับ - ครอบคลุมบริการด้านการแพทย์และสาธารณสุข การสร้างเสริมสุขภาพและการป้องกันโรค การตรวจวินิจฉัยโรค การรักษา และการฟื้นฟูสมรรถภาพ บริการการแพทย์แผนไทยและแพทย์ทางเลือก - ครอบคลุมรายการอุปกรณ์ อวัยวะเทียมในการบําบัดโรค ตามความจําเป็นอย่างทั่วถึง - ครอบคลุมบริการอุปกรณ์เครื่องช่วยสําหรับคนพิการ บริการฟื้นฟูสมรรถภาพด้านการแพทย์ระยะกลาง (Intermediate care :IMC) การฝึกทักษะการดํารงชีวิตอิสระสําหรับคนพิการทางร่างกายและการเคลื่อนไหว (Independent Living) ฝึกใช้อุปกรณ์เครื่องช่วยคนพิการ การฝึกทักษะการทําความคุ้นเคยกับสภาพแวดล้อม และการเคลื่อนไหว (Orientation & Mobility: Q&M) - ครอบคลุมบริการฟื้นฟูสมรรถภาพด้านการแพทย์ ผู้ป่วยที่จําเป็นต้องได้รับบริการฟื้นฟูสมรรถภาพด้านการแพทย์ 9 รายการ ได้แก่ 1.กายภาพบําบัด 2.กิจกรรมบําบัด 3.การแก้ไขการพูด 4.จิตบําบัด 5.พฤติกรรมบําบัด 6.ฟื้นฟ

In [22]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

embed = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
def embed_encode(docs):
    return embed.encode(docs)['dense_vecs']
scores = reranker.compute_score(data_list, normalize=True)
print(scores) 

KeyboardInterrupt: 